In [1]:
import pybedtools
import pandas as pd
from collections import defaultdict, Counter
import glob
from biodata.delimited import DelimitedWriter
import numpy as np
import sys
import h5py
import hdf5plugin
from biodatatools.utils.common import json_load
from pathlib import Path

In [2]:
sys.path.append(str(Path.cwd().parent))
import utils

In [3]:
PROJECT_DIR_s = "/fs/cbsuhy02/storage/yc2553/yc2553/projects/3.Human_atlas/procapnet/"
PROJECT_DIR_d = "/fs/cbsuhy02/storage/yc2553/yc2553/databases/"
PROJECT_DIR_o = "/home/yc2553/projects/HEA/output/"

In [4]:
sys.path.append(f"{PROJECT_DIR_s}2_train_models/")
from data_loading import extract_sequences

In [ ]:
### ExtDataFig.2d
## Goal: identify fraction of PRO-cap peaks with motifs contributing to transcritpion and fraction of ATAC-seq peaks with motifs contributing to accessibility. Here, motif instances attributed to a given motif were those Fi-NeMo instances mapped to the original TF-MoDISco patterns identified by the corresponding model. For motifs lacking an original TF-MoDISco pattern in one model, the Fi-NeMo instances mapped to the CWMs from the other model were used. This approach ensured that motif instances were assigned based on the exact motif patterns learned by each model whenever available, while allowing for matches to motifs not detected by TF-MoDISco in one model but present in the other.

## Transcription
# Regions of interest: PRO-cap peaks
# Contribution scores: ProCapNet model
# Motif CWM: both ProCapNet (prioritized) and ChromBPNet models

## Accessibility
# Regions of interest: ATAC-seq peaks; further divided into those with and without transcription
# Contribution scores: ChromBPNet model
# Motif CWM: both ProCapNet and ChromBPNet (prioritized) models

In [ ]:
### ExtDataFig.2e
## Goal: pairwise comparison of contribution scores to transcritpion and accessibility at the same motif instances

## Transcription
# Regions of interest: PRO-cap peaks
# Contribution scores: ProCapNet model
# Motif CWM: both ProCapNet (prioritized) and ChromBPNet models

## Accessibility
# Regions of interest: PRO-cap peaks (same input regions as transcription)
# Contribution scores: ChromBPNet model (recalculate the scores using the new input regions; different from the above analysis)
# Motif CWM: both ProCapNet and ChromBPNet (prioritized) models

# We compare the contribution scores from each model at individual motif instances; for the same motif type, we merge the motif boundary between motif calls from this two groups

# ProCapNet (PRO-cap peaks)

In [ ]:
# https://github.com/austintwang/finemo_gpu/issues/35

## Data preparation

In [5]:
# For file format requirement by FiNeMo: peak region coordinates in uncompressed ENCODE NarrowPeak format
# ENCODE narrowPeak: https://genome.ucsc.edu/FAQ/FAQformat.html#format12
# NARROWPEAK_SCHEMA = ["chr", "peak_start", "peak_end", "peak_name", "peak_score", 
#                      "peak_strand", "peak_signal", "peak_pval", "peak_qval", "peak_summit"]

s = "SC5"
inputfile = f"{PROJECT_DIR_o}procapnet/processed/{s}/peaks.bed.gz"
b1 = pybedtools.BedTool(inputfile)
outputfile = f"{PROJECT_DIR_o}procapnet/finemo/{s}/input.bed"
with DelimitedWriter(outputfile) as dw:
	for i in b1:
		chrom, start, end = i.fields
		# centered on the midpoint
		center = (int(end)-int(start))//2
		dw.write([chrom, start, end, "_".join([chrom, start, end])] + ["."]*5 + [center])

In [6]:
! head $outputfile -n 2

chr1	181449	181482	chr1_181449_181482	.	.	.	.	.	16
chr1	629905	629950	chr1_629905_629950	.	.	.	.	.	22


In [7]:
# Get one hot sequences

one_hot = extract_sequences(f"{PROJECT_DIR_s}genomes/hg38/hg38.withrDNA.fasta", 
							f"{PROJECT_DIR_s}genomes/hg38/hg38.withrDNA.chrom.sizes", 
							f"{PROJECT_DIR_o}procapnet/finemo/{s}/input.bed"
							)

Loading genome sequence from /fs/cbsuhy02/storage/yc2553/yc2553/projects/3.Human_atlas/procapnet/genomes/hg38/hg38.withrDNA.fasta
== In Extract Sequences ==
Peak filepath: /home/yc2553/projects/HEA/output/procapnet/finemo/SC5/input.bed
Sequence length: 2114
Num. Examples: 76899


In [8]:
outputfile = f"{PROJECT_DIR_o}procapnet/finemo/{s}/PROcap_onehot.npz"
np.savez_compressed(outputfile, one_hot)

## Preprocessing

In [ ]:
# Identify motif instances using FiNeMo
# https://github.com/austintwang/finemo_gpu

In [6]:
# Run this in "finemo" conda environment

model_type = "strand_merged_umap"
utils.finemo_preprocessing(f"{PROJECT_DIR_o}procapnet/finemo/{s}/PROcap_onehot.npz", 
			  f"{PROJECT_DIR_o}procapnet/deepshap_out/{s}/{model_type}/merged/all_counts_deepshap.npy", 
			  f"{PROJECT_DIR_o}procapnet/finemo/{s}/procapnet_scores/counts"
			  )

finemo extract-regions-modisco-fmt -s /home/yc2553/projects/HEA/output/procapnet/finemo/SC5/PROcap_onehot.npz -a /home/yc2553/projects/HEA/output/procapnet/deepshap_out/SC5/strand_merged_umap/merged/all_counts_deepshap.npy -o /home/yc2553/projects/HEA/output/procapnet/finemo/SC5/procapnet_scores/counts -w 1000


In [11]:
# Check the output

data = np.load(f"{PROJECT_DIR_o}procapnet/finemo/{s}/procapnet_scores/counts.npz")
data['sequences'].shape, data['contributions'].shape

((76899, 4, 1000), (76899, 4, 1000))

## Call hits

In [8]:
modisco = {"procapnet": f"{PROJECT_DIR_o}procapnet/modisco_out/{s}/{model_type}/merged/counts_modisco_results.hd5",
		   "chrombpnet": f"{PROJECT_DIR_d}papers/39829783/tfmodisco.raw_output.counts.GSE267154.hd5"
		  }

In [9]:
for k in modisco:
	utils.finemo_call_hits(f"{PROJECT_DIR_o}procapnet/finemo/{s}/procapnet_scores/counts.npz", 
			  modisco[k], 
			  f"{PROJECT_DIR_o}procapnet/finemo/{s}/procapnet_scores/{k}_motifs/",
			  f"{PROJECT_DIR_o}procapnet/finemo/{s}/input.bed"
			 )

finemo call-hits -r /home/yc2553/projects/HEA/output/procapnet/finemo/SC5/procapnet_scores/counts.npz -m /home/yc2553/projects/HEA/output/procapnet/modisco_out/SC5/strand_merged_umap/merged/counts_modisco_results.hd5 -o /home/yc2553/projects/HEA/output/procapnet/finemo/SC5/procapnet_scores/procapnet_motifs/ -p /home/yc2553/projects/HEA/output/procapnet/finemo/SC5/input.bed -a 0.7 -J
finemo call-hits -r /home/yc2553/projects/HEA/output/procapnet/finemo/SC5/procapnet_scores/counts.npz -m /fs/cbsuhy02/storage/yc2553/yc2553/databases/papers/39829783/tfmodisco.raw_output.counts.GSE267154.hd5 -o /home/yc2553/projects/HEA/output/procapnet/finemo/SC5/procapnet_scores/chrombpnet_motifs/ -p /home/yc2553/projects/HEA/output/procapnet/finemo/SC5/input.bed -a 0.7 -J


In [ ]:
# hits.tsv: may list multiple instances of a hit, each linked to a different peak.
# chr: Chromosome name. NA if peak coordinates (-p/--peaks) are not provided.
# start: Hit start coordinate from trimmed CWM, zero-indexed. Absolute if peak coordinates are provided, otherwise relative to the input region.
# end: Hit end coordinate from trimmed CWM, zero-indexed, exclusive. Absolute if peak coordinates are provided, otherwise relative to the input region.
# start_untrimmed: Hit start coordinate from trimmed CWM, zero-indexed. Absolute if peak coordinates are provided, otherwise relative to the input region.
# end_untrimmed: Hit end coordinate from trimmed CWM, zero-indexed,exclusive. Absolute if peak coordinates are provided, otherwise relative to the input region.
# motif_name: The hit motif name as specified in the provided tfmodisco H5 file.
# hit_coefficient: The regression coefficient for the hit, normalized per peak region.
# hit_coefficient_global: The regression coefficient for the hit, scaled by the overall importance of the region. This is the primary hit score.
# hit_correlation: The correlation between the untrimmed CWM and the contribution score of the motif hit.
# hit_importance: The total absolute contribution score within the motif hit.
# strand: The orientation of the hit (+ or -).
# peak_name: The name of the peak region containing the hit, taken from the name field of the input peak data. NA if -p/--peaks is not provided.
# peak_id: The numerical index of the peak region containing the hit.

In [18]:
# Check the output

df_hits = {}
df_hits["procapnet"] = pd.read_table(f"{PROJECT_DIR_o}procapnet/finemo/{s}/procapnet_scores/procapnet_motifs/hits.tsv")
df_hits["procapnet"].head(2)

,chr,start,end,start_untrimmed,end_untrimmed,motif_name,hit_coefficient,hit_coefficient_global,hit_correlation,hit_importance,strand,peak_name,peak_id
0,chr1,181096,181106,181087,181117,pos_patterns.pattern_0,0.014527,9.459502e-08,0.797305,0.017479,-,chr1_181449_181482,0
1,chr1,181096,181103,181088,181118,pos_patterns.pattern_5,0.859720,5.598366e-06,0.826864,0.014757,+,chr1_181449_181482,0


In [19]:
df_hits["chrombpnet"] = pd.read_table(f"{PROJECT_DIR_o}procapnet/finemo/{s}/procapnet_scores/chrombpnet_motifs/hits.tsv")

In [14]:
motifs = json_load(f"{PROJECT_DIR_o}procapnet/modisco_out/ESC_motifs.json")
mapping = defaultdict(dict)
for k in motifs:
	for motif in motifs[k]:
		pattern = f"pos_patterns.pattern_{motifs[k][motif]}"
		mapping[k][pattern] = motif

In [16]:
def merge_hits(dfs, prioritize, outputfile):
	dfs_filtered = {}
	for k in dfs:
		dfs[k]["motif"] = dfs[k]["motif_name"].map(mapping[k])
		dfs_filtered[k] = dfs[k].dropna(subset=["motif"])

	other = "chrombpnet" if prioritize == "procapnet" else "procapnet"
	df_other = dfs_filtered[other][~dfs_filtered[other]["motif"].isin(dfs_filtered[prioritize]["motif"])]

	df_combined = pd.concat([dfs_filtered[prioritize], df_other], ignore_index=True)
	df_combined.to_csv(outputfile, sep="\t", index=False)

In [20]:
# Merge motif hits
# Keep all motif types in procapnet; add those only found in chrombpnet

outputfile = f"{PROJECT_DIR_o}procapnet/finemo/{s}/procapnet_scores/merged_hits.tsv"
merge_hits(df_hits, "procapnet", outputfile)

# ChromBPNet (ATAC-seq peaks)

## Data preparation

In [38]:
# To be consistent with the file format for "finemo extract-regions-modisco-fmt"
# seq_contrib.counts.fold_mean.GSE267154.tar.gz: https://www.synapse.org/Synapse:syn63862717

inputfile = f"{PROJECT_DIR_d}papers/39829783/seq_contrib.counts.fold_mean.modisco_input.GSE267154.h5"
with h5py.File(inputfile, "r", libver="latest") as data:
	one_hot = np.array(data["raw"]["seq"])
	hypo_shap = np.array(data["shap"]["seq"])

In [39]:
one_hot.shape, hypo_shap.shape

((236571, 4, 2114), (236571, 4, 2114))

In [40]:
outputfile = f"{PROJECT_DIR_o}chrombpnet/finemo/seq_contrib.counts.fold_mean.modisco_input.GSE267154.onehot.npz"
np.savez_compressed(outputfile, one_hot)

In [41]:
outputfile = f"{PROJECT_DIR_o}chrombpnet/finemo/seq_contrib.counts.fold_mean.modisco_input.GSE267154.hypo.npz"
np.savez_compressed(outputfile, hypo_shap)

In [42]:
# Input bed regions corresponding to contribution file modisco_input.encid.h5

inputfile = f"{PROJECT_DIR_d}papers/39829783/logs.seq_contrib.counts.input_regions.modisco.GSE267154.bed.gz"
df = pd.read_table(inputfile, header=None)
df.shape

(236571, 11)

In [43]:
# This file has one additional column with peak boundary and subpeak summit to identify unique subpeaks
# Subpeaks of the same peak have different input sequences (centered on peak summit) in their one-hot encoded sequence file

df.head(2)

,0,1,2,3,4,5,6,7,8,9,10
0,chr1,100028967,100029416,Peak_89756,321,.,6.35697,32.13467,29.88145,246,chr1_100028967_100029416_246
1,chr1,100034578,100034763,Peak_124401,209,.,3.06090,20.91069,18.81874,91,chr1_100034578_100034763_91


In [44]:
# Drop the last column to match the required file format

outputfile = f"{PROJECT_DIR_o}chrombpnet/finemo/logs.seq_contrib.counts.input_regions.modisco.GSE267154.narrowPeak"
df[df.columns[:-1]].to_csv(outputfile, sep="\t", header=None, index=False)

## Preprocessing

In [45]:
utils.finemo_preprocessing(f"{PROJECT_DIR_o}chrombpnet/finemo/seq_contrib.counts.fold_mean.modisco_input.GSE267154.onehot.npz", 
			  f"{PROJECT_DIR_o}chrombpnet/finemo/seq_contrib.counts.fold_mean.modisco_input.GSE267154.hypo.npz", 
			  f"{PROJECT_DIR_o}chrombpnet/finemo/counts.GSE267154"
			  )

finemo extract-regions-modisco-fmt -s /home/yc2553/projects/HEA/output/chrombpnet/finemo/seq_contrib.counts.fold_mean.modisco_input.GSE267154.onehot.npz -a /home/yc2553/projects/HEA/output/chrombpnet/finemo/seq_contrib.counts.fold_mean.modisco_input.GSE267154.hypo.npz -o /home/yc2553/projects/HEA/output/chrombpnet/finemo/counts.GSE267154 -w 1000


In [46]:
# Check the output

data = np.load(f"{PROJECT_DIR_o}chrombpnet/finemo/counts.GSE267154.npz")
data['sequences'].shape, data['contributions'].shape

((236571, 4, 1000), (236571, 4, 1000))

## Call hits

In [11]:
for k in modisco:
	utils.finemo_call_hits(f"{PROJECT_DIR_o}chrombpnet/finemo/counts.GSE267154.npz", 
			  modisco[k], 
			  f"{PROJECT_DIR_o}chrombpnet/finemo/{k}_motifs/",
			  f"{PROJECT_DIR_o}chrombpnet/finemo/logs.seq_contrib.counts.input_regions.modisco.GSE267154.narrowPeak"
			 )

finemo call-hits -r /home/yc2553/projects/HEA/output/chrombpnet/finemo/counts.GSE267154.npz -m /home/yc2553/projects/HEA/output/procapnet/modisco_out/SC5/strand_merged_umap/merged/counts_modisco_results.hd5 -o /home/yc2553/projects/HEA/output/chrombpnet/finemo/procapnet_motifs/ -p /home/yc2553/projects/HEA/output/chrombpnet/finemo/logs.seq_contrib.counts.input_regions.modisco.GSE267154.narrowPeak -a 0.7 -J
finemo call-hits -r /home/yc2553/projects/HEA/output/chrombpnet/finemo/counts.GSE267154.npz -m /fs/cbsuhy02/storage/yc2553/yc2553/databases/papers/39829783/tfmodisco.raw_output.counts.GSE267154.hd5 -o /home/yc2553/projects/HEA/output/chrombpnet/finemo/chrombpnet_motifs/ -p /home/yc2553/projects/HEA/output/chrombpnet/finemo/logs.seq_contrib.counts.input_regions.modisco.GSE267154.narrowPeak -a 0.7 -J


In [21]:
df_hits = {}
for k in modisco:
	df_hits[k] = pd.read_table(f"{PROJECT_DIR_o}chrombpnet/finemo/{k}_motifs/hits.tsv")
outputfile = f"{PROJECT_DIR_o}chrombpnet/finemo/merged_hits.tsv"
merge_hits(df_hits, "chrombpnet", outputfile)

# ChromBPNet (PRO-cap peaks)

## Data preparation

In [ ]:
# Get contribution scores from ChromBPNet using the same input region from ProCapNet
# https://github.com/kundajelab/chrombpnet/wiki/Generate-contribution-score-bigwigs
# It's recommnded to use the chrombpnet_nobias.h5 model for this.

In [50]:
# models.GSE267154.tar.gz: https://www.synapse.org/Synapse:syn63862585
# Add gpu at the end

script = f"{PROJECT_DIR_s}slurm/deepshap.sh"
regions = f"{PROJECT_DIR_o}procapnet/finemo/{s}/input.bed"
task = "counts"
for fold in range(5):
	model = f"{PROJECT_DIR_d}papers/39829783/models/model.chrombpnet_nobias.fold_{fold}.GSE267154.h5"
	output_prefix = f"{PROJECT_DIR_o}procapnet/finemo/{s}/chrombpnet_scores/contrib/fold_{fold}.GSE267154"
	commands = ["sbatch", script,
				model,
				regions,
				output_prefix,
				task
				]
	print(" ".join(commands))

sbatch /fs/cbsuhy02/storage/yc2553/yc2553/projects/3.Human_atlas/procapnet/slurm/deepshap.sh /fs/cbsuhy02/storage/yc2553/yc2553/databases/papers/39829783/models/model.chrombpnet_nobias.fold_0.GSE267154.h5 /home/yc2553/projects/HEA/output/procapnet/finemo/SC5/input.bed /home/yc2553/projects/HEA/output/procapnet/finemo/SC5/chrombpnet_scores/contrib/fold_0.GSE267154 counts
sbatch /fs/cbsuhy02/storage/yc2553/yc2553/projects/3.Human_atlas/procapnet/slurm/deepshap.sh /fs/cbsuhy02/storage/yc2553/yc2553/databases/papers/39829783/models/model.chrombpnet_nobias.fold_1.GSE267154.h5 /home/yc2553/projects/HEA/output/procapnet/finemo/SC5/input.bed /home/yc2553/projects/HEA/output/procapnet/finemo/SC5/chrombpnet_scores/contrib/fold_1.GSE267154 counts
sbatch /fs/cbsuhy02/storage/yc2553/yc2553/projects/3.Human_atlas/procapnet/slurm/deepshap.sh /fs/cbsuhy02/storage/yc2553/yc2553/databases/papers/39829783/models/model.chrombpnet_nobias.fold_2.GSE267154.h5 /home/yc2553/projects/HEA/output/procapnet/finemo

In [51]:
# Get average scores

all_scores = []
for fold in range(5):
	inputfile = f"{PROJECT_DIR_o}procapnet/finemo/{s}/chrombpnet_scores/contrib/fold_{fold}.GSE267154.counts_scores.h5"
	with h5py.File(inputfile, "r", libver="latest") as data:
		projected_shap = np.array(data["projected_shap"]["seq"])
		all_scores.append(projected_shap)
mean = np.array(all_scores).mean(axis=0)
outputfile = f"{PROJECT_DIR_o}procapnet/finemo/{s}/chrombpnet_scores/contrib/GSE267154.counts_scores.mean.npy"
np.save(outputfile, mean)

## Preprocessing

In [32]:
utils.finemo_preprocessing(f"{PROJECT_DIR_o}procapnet/finemo/{s}/PROcap_onehot.npz", 
			  f"{PROJECT_DIR_o}procapnet/finemo/{s}/chrombpnet_scores/contrib/GSE267154.counts_scores.mean.npy", 
			  f"{PROJECT_DIR_o}procapnet/finemo/{s}/chrombpnet_scores/counts"
			 )

finemo extract-regions-modisco-fmt -s /home/yc2553/projects/HEA/output/procapnet/finemo/SC5/PROcap_onehot.npz -a /home/yc2553/projects/HEA/output/procapnet/finemo/SC5/chrombpnet/contrib/GSE267154.counts_scores.mean.npy -o /home/yc2553/projects/HEA/output/procapnet/finemo/SC5/chrombpnet/PROcap_counts -w 1000


In [33]:
# Check the output

data = np.load(f"{PROJECT_DIR_o}procapnet/finemo/{s}/chrombpnet_scores/counts.npz")
data['sequences'].shape, data['contributions'].shape

((76899, 4, 1000), (76899, 4, 1000))

## Call hits

In [12]:
for k in modisco:
	utils.finemo_call_hits(f"{PROJECT_DIR_o}procapnet/finemo/{s}/chrombpnet_scores/counts.npz", 
			  modisco[k],
			  f"{PROJECT_DIR_o}procapnet/finemo/{s}/chrombpnet_scores/{k}_motifs/",
			  f"{PROJECT_DIR_o}procapnet/finemo/{s}/input.bed"
			 )

finemo call-hits -r /home/yc2553/projects/HEA/output/procapnet/finemo/SC5/chrombpnet_scores/counts.npz -m /home/yc2553/projects/HEA/output/procapnet/modisco_out/SC5/strand_merged_umap/merged/counts_modisco_results.hd5 -o /home/yc2553/projects/HEA/output/procapnet/finemo/SC5/chrombpnet_scores/procapnet_motifs/ -p /home/yc2553/projects/HEA/output/procapnet/finemo/SC5/input.bed -a 0.7 -J
finemo call-hits -r /home/yc2553/projects/HEA/output/procapnet/finemo/SC5/chrombpnet_scores/counts.npz -m /fs/cbsuhy02/storage/yc2553/yc2553/databases/papers/39829783/tfmodisco.raw_output.counts.GSE267154.hd5 -o /home/yc2553/projects/HEA/output/procapnet/finemo/SC5/chrombpnet_scores/chrombpnet_motifs/ -p /home/yc2553/projects/HEA/output/procapnet/finemo/SC5/input.bed -a 0.7 -J


In [22]:
df_hits = {}
for k in modisco:
	df_hits[k] = pd.read_table(f"{PROJECT_DIR_o}procapnet/finemo/{s}/chrombpnet_scores/{k}_motifs/hits.tsv")
outputfile = f"{PROJECT_DIR_o}procapnet/finemo/{s}/chrombpnet_scores/merged_hits.tsv"
merge_hits(df_hits, "chrombpnet", outputfile)